In [1]:
import os
import cv2
import sys
from PyQt5.QtCore import Qt
from PyQt5.uic import loadUi
from PyQt5 import QtWidgets, QtGui, QtCore
from PyQt5.QtCore import QCoreApplication
from PyQt5.QtGui import QPixmap, QPixmapCache, QImage
from PyQt5.QtWidgets import QDialog, QApplication, QWidget, QMainWindow, QFileDialog, QAction
        
def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder,filename))
        if img is not None:
            images.append(img)
    return images

def to_gray_image(filepath):
    image = cv2.imread(filepath)
    return cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

def to_gray_folder(filepath):
    images = load_images_from_folder(filepath)
    images_gray = []
    for image in images:
        images_gray.append(cv2.cvtColor(image, cv2.COLOR_BGR2GRAY))
    return images_gray

class WelcomeScreen(QMainWindow):
    def __init__(self):
        super(WelcomeScreen,self).__init__()
        loadUi("page1.ui",self)
        pixmap = QPixmap('logo(2).png')
        self.label.setPixmap(pixmap)
        self.label_2.setPixmap(QPixmap('wallpaper.jpg'))
        self.start_btn.clicked.connect(lambda: self.start_click())
    
    def start_click(self):
        select_file = SelectScreen() 
        widget.addWidget(select_file)
        widget.setCurrentIndex(widget.currentIndex()+1)

class SelectScreen(QMainWindow):
    def __init__(self):
        super(SelectScreen,self).__init__()
        loadUi("page2.ui",self)
        self.setAcceptDrops(True)
        self.label_2.setPixmap(QPixmap('wallpaper.jpg'))
        self.pushFolderButton.clicked.connect(lambda: self.open_folder())
        self.pushImageButton.clicked.connect(lambda: self.open_image())
    
    def open_folder(self):
        file = QtWidgets.QFileDialog.getExistingDirectory(self, 'Select Folder')
        if os.path.exists(file):
            select_file = FolderPretrainScreen(file) 
            widget.addWidget(select_file)
            widget.setCurrentIndex(widget.currentIndex()+1)
        
    def open_image(self):
        file = QtWidgets.QFileDialog.getOpenFileName(self, "Select Image", None, "Image (*.png *.jpg)")[0]
        if "jpg" in file or "png" in file:
            select_file = ImagePretrainScreen(file) 
            widget.addWidget(select_file)
            widget.setCurrentIndex(widget.currentIndex()+1)
        
    def dragEnterEvent(self, event):
        if event.mimeData().hasImage:
            event.accept()
        else:
            event.ignore()

    def dragMoveEvent(self, event):
        if event.mimeData().hasImage:
            event.accept()
        else:
            event.ignore()

    def dropEvent(self, event):
        if event.mimeData().hasImage:
            event.setDropAction(Qt.CopyAction)
            self.file_path = event.mimeData().urls()[0].toLocalFile()
            self.set_image(self.file_path)
            event.accept()
        else:
            event.ignore()

    def set_image(self, file):
        if "jpg" in file or "png" in file:
            select_file = ImagePretrainScreen(file) 
            widget.addWidget(select_file)
            widget.setCurrentIndex(widget.currentIndex()+1)
        else:
            select_file = FolderPretrainScreen(file) 
            widget.addWidget(select_file)
            widget.setCurrentIndex(widget.currentIndex()+1)

class ImagePretrainScreen(QMainWindow):
    def __init__(self, file):
        super(ImagePretrainScreen,self).__init__()
        loadUi("page3_image.ui",self)
        pixmap = QPixmap(file)
        self.label_2.setPixmap(QPixmap('wallpaper.jpg'))
        self.label.setPixmap(pixmap.scaled(self.label.size(), QtCore.Qt.KeepAspectRatio))
        print(file)
        
        self.toNeuralButton.clicked.connect(lambda: self.findDrops(file))
        self.ImageProcessingButton.clicked.connect(lambda: self.ImageProcessing(file))
        self.BackButton.clicked.connect(lambda: self.Back())
    
    def findDrops(self, file):
        select_file = ImageResultScreen(file) 
        widget.addWidget(select_file)
        widget.setCurrentIndex(widget.currentIndex()+1)
        
    def ImageProcessing(self, file):
        select_file = EditScreen(file)
        widget.addWidget(select_file)
        widget.setCurrentIndex(widget.currentIndex()+1)
        
    def Back(self):
        select_file = SelectScreen()
        widget.addWidget(select_file)
        widget.setCurrentIndex(widget.currentIndex()+1)

class FolderPretrainScreen(QMainWindow):
    def __init__(self, file):
        super(FolderPretrainScreen,self).__init__()
        loadUi("page3_folder.ui",self)
        self.label.setText("Your choice: \n" + file)
        self.label_2.setPixmap(QPixmap('wallpaper.jpg'))
        print(file)
        
        self.toNeuralButton.clicked.connect(lambda: self.findDrops(file))
        self.BackButton.clicked.connect(lambda: self.Back())
    
    def findDrops(self, file):
        select_file = FolderResultScreen(file) 
        widget.addWidget(select_file)
        widget.setCurrentIndex(widget.currentIndex()+1)
        
    def Back(self):
        select_file = SelectScreen() 
        widget.addWidget(select_file)
        widget.setCurrentIndex(widget.currentIndex()+1)

class EditScreen(QMainWindow):
    def __init__(self, file):
        super(EditScreen,self).__init__()
        loadUi("page4_image.ui",self)
        self.photos = [QPixmap(file)]
        self.rotation = 0
        self.pixmap = QPixmap(file)
        self.edit_pixmap = QPixmap(file)
        self.label_3.setAlignment(QtCore.Qt.AlignLeft | QtCore.Qt.AlignTop)
        self.label_3.setPixmap(QPixmap('wallpaper.jpg'))
        self.label.setPixmap(QPixmap(file).scaled(self.label.size(), QtCore.Qt.KeepAspectRatio))
        
        self.okButton.clicked.connect(lambda: self.ok(file))
        self.backButton.clicked.connect(lambda: self.Back())
        self.rotateButton.clicked.connect(lambda: self.rotate())
 
    def ok(self, file):
        self.photos[-1].save('pixmap.png')
        select_file = ImageResultScreen(file) 
        widget.addWidget(select_file)
        widget.setCurrentIndex(widget.currentIndex()+1)
        
    def Back(self):
        if(len(self.photos)>1):
            self.photos.pop()
            self.label.setPixmap(self.photos[-1].scaled(self.label.size(), QtCore.Qt.KeepAspectRatio))
    
    def rotate(self):
        pixmap = self.edit_pixmap.copy()
        self.rotation += 90
        transform = QtGui.QTransform().rotate(90)
        pixmap = pixmap.transformed(transform, QtCore.Qt.SmoothTransformation)
        self.label.setPixmap(pixmap.scaled(self.label.size(), QtCore.Qt.KeepAspectRatio))
        self.edit_pixmap = pixmap
        self.photos.append(pixmap)
        pixmap.save('pixmap.png')
        
    def mousePressEvent (self, eventQMouseEvent):
        offset = self.label.pos()
        self.label.originQPoint = eventQMouseEvent.pos()-offset
        self.label.currentQRubberBand = QtWidgets.QRubberBand(QtWidgets.QRubberBand.Rectangle, self.label)
        self.label.currentQRubberBand.setGeometry(QtCore.QRect(self.label.originQPoint, QtCore.QSize()))
        self.label.currentQRubberBand.show()

    def mouseMoveEvent (self, eventQMouseEvent):
        offset = self.label.pos()
        self.label.currentQRubberBand.setGeometry(QtCore.QRect(self.label.originQPoint, eventQMouseEvent.pos()-offset).normalized())

    def mouseReleaseEvent (self, eventQMouseEvent):
        self.label.currentQRubberBand.hide()
        currentQRect = self.label.currentQRubberBand.geometry()
        self.label.currentQRubberBand.deleteLater()
        self.edit_pixmap = self.label.grab(currentQRect)
        self.photos.append(self.edit_pixmap)
        self.label.setPixmap(self.edit_pixmap.scaled(self.label.size(), QtCore.Qt.KeepAspectRatio))
        self.edit_pixmap.save('pixmap.png')

        
class ImageResultScreen(QMainWindow):
    def __init__(self, file):
        super(ImageResultScreen,self).__init__()
        loadUi("page5_image.ui",self)
        self.label_3.setPixmap(QPixmap('wallpaper.jpg'))
        if os.path.exists("pixmap.png"):
            result_img = to_gray_image("pixmap.png")
            os.remove("pixmap.png")
        else:
            result_img = to_gray_image(file)
        
        if len(result_img.shape)<3:
            frame = cv2.cvtColor(result_img, cv2.COLOR_GRAY2RGB)
        else:
            frame = cv2.cvtColor(result_img, cv2.COLOR_BGR2RGB)
            
        h, w = result_img.shape[:2]
        bytesPerLine = 3 * w
        qimage = QImage(frame.data, w, h, bytesPerLine, QImage.Format.Format_RGB888) 
        pix = QPixmap(qimage)
        self.label.setPixmap(pix.scaled(self.label.size(), QtCore.Qt.KeepAspectRatio))
        
        self.saveButton.clicked.connect(lambda: self.savePhoto(result_img))
        self.quitButton.clicked.connect(lambda: QApplication.closeAllWindows())

    def savePhoto(self, image):
        filename = QFileDialog.getSaveFileName(filter="JPG(*.jpg);;PNG(*.png)")[0]
        print('Image saved as:',filename)
        if(filename):
            cv2.imwrite(filename, image)


class FolderResultScreen(QMainWindow):
    def __init__(self, file):
        super(FolderResultScreen,self).__init__()
        loadUi("page5_folder.ui",self)
        self.label_4.setPixmap(QPixmap('wallpaper.jpg'))
        result_img = to_gray_folder(file)
        
        self.index = 0
        
        if len(result_img[self.index].shape)<3:
            frame = cv2.cvtColor(result_img[self.index], cv2.COLOR_GRAY2RGB)
        else:
            frame = cv2.cvtColor(result_img[self.index], cv2.COLOR_BGR2RGB)
            
        h, w = result_img[self.index].shape[:2]
        bytesPerLine = 3 * w
        qimage = QImage(frame.data, w, h, bytesPerLine, QImage.Format.Format_RGB888) 
        pix = QPixmap(qimage)
        self.label.setPixmap(pix.scaled(self.label.size(), QtCore.Qt.KeepAspectRatio))
        
        if(len(result_img) == 2):
            self.label_3.setText("and "+str(len(result_img)-1)+" more image")
        if(len(result_img) > 2):
            self.label_3.setText("and "+str(len(result_img)-1)+" more images")
            
        self.saveButton.clicked.connect(lambda: self.savePhoto(result_img))
        self.quitButton.clicked.connect(lambda: QApplication.closeAllWindows())

    def savePhoto(self, images):
        folder = QtWidgets.QFileDialog.getExistingDirectory(self, 'Select Folder')
        print(folder)
        if os.path.exists(folder):
            for i in range(len(images)):
                cv2.imwrite(folder+"/result"+str(i)+".png", images[i])
                print('Image saved as:',folder+"/result"+str(i)+".png")
                
            
# main
app = QCoreApplication.instance()
if app is None:
    app = QApplication(sys.argv)
    welcome = WelcomeScreen()
    widget = QtWidgets.QStackedWidget()
    widget.addWidget(welcome)
    widget.setFixedHeight(590)
    widget.setFixedWidth(800)
    widget.show()
try:
    sys.exit(app.exec_())
except:
    print("Exiting")

C:/Users/USER/Downloads/drop/output.png
Image saved as: 
Exiting
